In [33]:
import os
from typing import Annotated, Optional

import httpx
import instructor
from dotenv import find_dotenv, load_dotenv
from models import *
from openai import OpenAI
from prompts import *
from pydantic import AfterValidator, BaseModel, BeforeValidator, Field
from rich import print

%load_ext autoreload
%autoreload 2

%load_ext rich

load_dotenv(find_dotenv())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rich extension is already loaded. To reload it, use:
  %reload_ext rich


True

In [18]:
llm_client = instructor.patch(
    OpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
    ),
)

In [50]:
job_description = """
About the Role

We are looking for someone who enjoys rolling up their sleeves to join our New Verticals rocketship on Drive - DoorDash’s white label fulfillment platform and product - to scale business with strategic partners, unlock accessibility to local businesses on their own terms, and launch new products and services to expand our TAM.

The Associate Manager, Drive New Verticals S&O Canada, will oversee the growth, profitability and operations strategy for our Canada business.

You are a first-principles thinker who is both a strategist and a builder. Reporting to the Manager of Drive Strategy and Operations Canada, you will work closely with product, engineering, sales, operations, business development and other teams to make your vision a reality.

We expect this role to be hybrid with some time in-office and some time remote.

You’re excited about this opportunity because you will
• Strategize : You will develop the strategy to best serve our customers, thinking at a deeply strategic level about the success now and 5-10+ years out.
• Operate : You want to get to the lowest level of detail, and scale. You will work directly with internal and external partners to execute on your short & long-term plans;

you are a cross functional success enabler and an excellent operator.
• Build : You will develop and lead 0 to 1 opportunities, creating the roadmap for future growth vectors and determine how to invest our resources to achieve desired outcomes.
• Collaborate : You value partnerships, and will excel at working across Operations, Product, Sales, Engineering, and more to bring new products and services to merchants.

We’re excited about you because
• You are an owner. You display extreme ownership across every workstream while ruthlessly prioritizing what will actually move the needle
• You solve problems from first principles. You’re excited to solve problems in innovative ways, and you resist this is how it’s always been done thinking.
• You thrive with a diverse set of responsibilities. You love working across a breadth of disciplines and teams.
• You’re an analytical powerhouse. You let data win arguments, and you’re comfortable pulling your own data in SQL and modeling in Excel.
• You’re persuasive. You are an effective communicator, speaking as if you’re right and listening as if you’re wrong. You meet people on their level.
• You act with urgency. Speed excites you. You prefer an accomplishment today to an accomplishment tomorrow.
• You’re relentless. You do what it takes to win, no matter what.
• You are a builder. You enjoy building from the ground up and excel in creating structure and process.You don’t need a guide.

You dive into difficult problems and come up with a plan.

You have 4-6 years of experience. You’ve succeeded in a competitive, challenging environment.

Bonus Points
• You have experience with and are fluent in SQL and excel.
• You have experience in managing or supporting a complex P&L.
• You have experience launching new partners or new products.
• You have a demonstrated interest in logistics, on-demand services, and marketplaces.
• You have experience in project management.

Why You’ll Love Working at DoorDash
• We are leaders - Leadership is not limited to our management team. It’s something everyone at DoorDash embraces and embodies.
• We are doers - We believe the only way to predict the future is to build it. Creating solutions that will lead our company and our industry is what we do on every project, every day.
• We are learners - We’re not afraid to dig in and uncover the truth, even if it’s scary or inconvenient. Everyone here is continually learning on the job, no matter if we’ve been in a role for one year or one minute.
• We are customer-obsessed - Our mission is to grow and empower local economies. We are committed to our customers, merchants, and dashers and believe in connecting people with possibility.
• We are all DoorDash - The magic of DoorDash is our people, together making our inspiring goals attainable and driving us to greater heights.
• We offer great compensation packages and comprehensive health benefits
"""

In [60]:
api_ = httpx.Client(base_url="http://0.0.0.0:8000", timeout=1200)


payload = RequestPayload(job_description=job_description, model='gpt-4-turbo-preview')
payload


RequestPayload(
    job_description='\nAbout the Role\n\nWe are looking for someone who enjoys rolling up their sleeves to join our New Verticals rocketship on Drive - DoorDash’s white label fulfillment platform and product - to scale business with strategic partners, unlock accessibility to local businesses on their own terms, and launch new products and services to expand our TAM.\n\nThe Associate Manager, Drive New Verticals S&O Canada, will oversee the growth, profitability and operations strategy for our Canada business.\n\nYou are a first-principles thinker who is both a strategist and a builder. Reporting to the Manager of Drive Strategy and Operations Canada, you will work closely with product, engineering, sales, operations, business development and other teams to make your vision a reality.\n\nWe expect this role to be hybrid with some time in-office and some time remote.\n\nYou’re excited about this opportunity because you will\n• Strategize : You will develop the strategy 

In [61]:
response = api_.post("/jd/", data=payload.model_dump_json())

In [62]:
response.json()


{
    'title': 'Associate Manager, Drive New Verticals S&O Canada',
    'company_information': 'DoorDash is a technology company that connects people with the best in their cities. We do this by empowering local businesses and in turn, generate new ways for people to earn, work and live. We started by facilitating door-to-door delivery, but we see this as just the beginning of connecting people with possibility — easier evenings, happier days, bigger savings accounts, wider nets and stronger communities.',
    'industry': 'Technology',
    'location': 'Canada',
    'job_type': 'Hybrid',
    'years_of_experience_required': 4,
    'years_of_experience_preferred': 6,
    'education': [{'level': 'Not Specified', 'field_of_study': None}],
    'salary_range': None,
    'responsibilities': [
        {'index': 1, 'description': 'Develop strategy to best serve customers'},
        {
            'index': 2,
            'description': 'Work directly with internal and external partners to execute

## Extract keywords

In [63]:
responsibilities = response.json()['responsibilities']
responsibilities


[
    {'index': 1, 'description': 'Develop strategy to best serve customers'},
    {'index': 2, 'description': 'Work directly with internal and external partners to execute plans'},
    {'index': 3, 'description': 'Develop and lead 0 to 1 opportunities for future growth'},
    {
        'index': 4,
        'description': 'Collaborate across Operations, Product, Sales, Engineering, and more'
    }
]

In [65]:
payload_keyword = RequestPayload(
    job_description=str(responsibilities),
    model="gpt-4-turbo-preview",
)

In [67]:
response_keyword = api_.post("/keywords/", data=payload_keyword.model_dump_json())

response_keyword.json()


{
    'keywords': [
        {
            'keyword': 'strategy development',
            'source': 'Develop strategy to best serve customers',
            'reasoning': 'Shows ability to plan and strategize for customer satisfaction.'
        },
        {
            'keyword': 'customer service',
            'source': 'Develop strategy to best serve customers',
            'reasoning': 'Indicates focus on serving and understanding customer needs.'
        },
        {
            'keyword': 'partnership management',
            'source': 'Work directly with internal and external partners to execute plans',
            'reasoning': 'Demonstrates ability to collaborate and manage relationships with various stakeholders.'
        },
        {
            'keyword': 'project execution',
            'source': 'Work directly with internal and external partners to execute plans',
            'reasoning': 'Highlights experience in implementing and executing plans.'
        },
        {
      